In [11]:
import numpy as np
def true_f(x: np.ndarray) -> np.ndarray:
    return x[0] + np.sin(x[1]) / 5


TEST_SIZE = 10_000
TRAIN_SIZE = 1000

x_validation = np.vstack(
    [
        np.random.random_sample(size=TEST_SIZE) * 2 * np.pi - np.pi,
        np.random.random_sample(size=TEST_SIZE) * 2 - 1,
    ]
)
y_validation = true_f(x_validation)

print(x_validation.shape)
print(y_validation.shape)
train_indexes = np.random.choice(TEST_SIZE, size=TRAIN_SIZE, replace=False)
x_train = x_validation[:, train_indexes]
y_train = y_validation[train_indexes]
assert np.all(y_train == true_f(x_train)), "D'ho"
np.savez('problem_0.npz', x=x_train, y=y_train)

(2, 10000)
(10000,)


In [13]:
import numpy as np
import random
from typing import Callable

# Define constants for the genetic programming algorithm
POPULATION_SIZE = 100
GENERATIONS = 50
TOURNAMENT_SIZE = 5
MUTATION_RATE = 0.2
CROSSOVER_RATE = 0.5
def safe_divide(a, b, eps=1e-12):
    b_safe = np.where(np.abs(b) > eps, b, eps * np.sign(b))  # Replace zeros with small values
    return a / b_safe



def safe_log(x, eps=1e-12):
    return np.where(x > eps, np.log(x), 0)

# Define the set of allowed numpy functions
FUNCTION_SET = [np.add, np.subtract, np.multiply, safe_divide,] #, np.exp, safe_log]
TERMINAL_SET = []  # This will be populated dynamically with input variables

# Load data from the .npz file
def load_data(file_path: str):
    try:
        data = np.load(file_path)
        x = data['x']
        y = data['y']
        return x, y
    except Exception as e:
        raise ValueError(f"Error loading data from {file_path}: {e}")

# Define a node class for the genetic programming tree
class Node:
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

    def is_operator(self):
        return self.value in FUNCTION_SET

    def is_variable(self):
        return isinstance(self.value, int)

    def is_constant(self):
        return isinstance(self.value, float)

    def evaluate(self, variables: np.ndarray):
        try:
            if callable(self.value):  # Function node
                left_val = self.left.evaluate(variables) if self.left else None
                right_val = self.right.evaluate(variables) if self.right else None
                
                # Handle divide-by-zero and log of non-positive values
                if self.value == np.divide and (right_val == 0 or np.isinf(right_val)):
                    return np.inf
                if self.value == np.log and (left_val <= 0 or np.isinf(left_val)):
                    return np.inf

                return self.value(left_val, right_val)
            elif isinstance(self.value, int):  # Variable index
                if 0 <= self.value < len(variables):  # Verifica che l'indice sia valido
                    return variables[self.value]
                else:
                    raise ValueError(f"Invalid variable index {self.value} for input {variables}")

            else:  # Constant
                return self.value
        except:
            return np.inf


    def __str__(self):
        if self.is_operator():
            return f"({str(self.left)} {self.value} {str(self.right)})"
        return str(self.value)

# Generate a random tree for initial population
def generate_random_tree(depth: int, num_variables: int, used_variables=None):
    if used_variables is None:
        used_variables = set()

    if depth == 0 or (depth > 1 and random.random() < 0.3 and len(used_variables) == num_variables):  # 30% chance to stop at terminal
        if random.random() < 0.5 and len(used_variables) < num_variables:  # Variable
            var_index = random.choice([i for i in range(num_variables) if i not in used_variables])
            used_variables.add(var_index)
            return Node(var_index)
        else:  # Constant
            return Node(random.uniform(-1.0, 1.0))
    else:
        func = random.choice(FUNCTION_SET)
        left = generate_random_tree(depth - 1, num_variables, used_variables)
        right = generate_random_tree(depth - 1, num_variables, used_variables)
        return Node(func, left, right)



# Evaluate fitness (mean squared error)
def fitness(tree: Node, x: np.ndarray, y: np.ndarray):
    try:
        # print(f"x shape: {x.shape}")
        # for xi in x.T:
        #     print("Input xi:", xi)
        #     result = tree.evaluate(xi)
        #     print("Tree: ", tree)
        #     print("Result:", result)

        predictions = np.array([tree.evaluate(xi) for xi in x.T])

        # print(f"Predictions: {predictions}")


        # single_sample = x[:, 0]  # First column, shape (2,)
        # print("Single sample:", single_sample)
        # result = tree.evaluate(single_sample)
        
        # print("Result:", result)

        # print(f"Fitness: {np.mean((predictions - y) ** 2)}")
        if np.any(np.isinf(predictions)) or np.any(np.isnan(predictions)):
            return np.inf
        return np.mean((predictions - y) ** 2)
    except:
        return np.inf



# Tournament selection
def tournament_selection(population, scores):
    tournament = random.sample(list(zip(population, scores)), TOURNAMENT_SIZE)
    tournament.sort(key=lambda ind: ind[1])
    return tournament[0][0]

# Crossover between two parent trees
def crossover(tree1: Node, tree2: Node):
    if tree1.is_operator() and tree2.is_operator():
        if random.random() < 0.5:
            return Node(tree1.value, crossover(tree1.left, tree2.left), crossover(tree1.right, tree2.right))
        else:
            return Node(tree2.value, crossover(tree1.left, tree2.left), crossover(tree1.right, tree2.right))
    elif tree1.is_operator():
        return Node(tree1.value, crossover(tree1.left, tree2), crossover(tree1.right, tree2))
    elif tree2.is_operator():
        return Node(tree2.value, crossover(tree1, tree2.left), crossover(tree1, tree2.right))
    else:
        return tree1 if random.random() < 0.5 else tree2

def validate_tree(tree: Node, num_variables: int):
    used_variables = set()
    def traverse(node):
        if node.is_variable():
            used_variables.add(node.value)
        if node.left:
            traverse(node.left)
        if node.right:
            traverse(node.right)
    traverse(tree)
    return len(used_variables) == num_variables


def mutate(tree: Node, num_variables: int):
    if random.random() < MUTATION_RATE:
        return generate_random_tree(random.randint(1, 3), num_variables)
    
    if tree.is_operator():
        if tree.left:
            tree.left = mutate(tree.left, num_variables)
        if tree.right:
            tree.right = mutate(tree.right, num_variables)
    else:
        if tree.left:
            tree.left = mutate(tree.left, num_variables)
        if tree.right:
            tree.right = mutate(tree.right, num_variables)
    
    return tree


# Main genetic programming loop
# def genetic_programming(x: np.ndarray, y: np.ndarray):
#     num_variables = x.shape[1]
#     TERMINAL_SET.extend(range(num_variables))

#     # Initialize population
#     population = [generate_random_tree(3, num_variables) for _ in range(POPULATION_SIZE)]

#     for generation in range(GENERATIONS):
#         scores = [fitness(ind, x, y) for ind in population]
#         new_population = []

#         while len(new_population) < POPULATION_SIZE:
#             parent1 = tournament_selection(population, scores)
#             parent2 = tournament_selection(population, scores)
#             child1, child2 = crossover(parent1, parent2)
#             child1 = mutate(child1, num_variables)
#             child2 = mutate(child2, num_variables)
#             new_population.extend([child1, child2])

#         population = new_population[:POPULATION_SIZE]

#         # **Population diversity check**
#         if len(set(str(ind) for ind in population)) < POPULATION_SIZE // 2:
#             # Reintroduce random individuals to increase diversity
#             population.extend([generate_random_tree(3, num_variables) for _ in range(POPULATION_SIZE // 2)])
#             population = population[:POPULATION_SIZE]  # Ensure population size remains constant

#         # Print best fitness of the generation
#         best_score = min(scores)
#         print(f"Generation {generation}: Best fitness = {best_score}")



#     scores = [fitness(ind, x, y) for ind in population]  # Recalculate scores for the final population
#     best_index = scores.index(min(scores))

    
#     return population[best_index]
def genetic_programming(x: np.ndarray, y: np.ndarray):
    num_variables = x.shape[0]
    print("Num variables:", num_variables)

    # Initialize population
    population = [generate_random_tree(3, num_variables) for _ in range(POPULATION_SIZE)]

    for generation in range(GENERATIONS):
        scores = [fitness(ind, x, y) for ind in population]
        new_population = []

        while len(new_population) < POPULATION_SIZE:
            parent1 = tournament_selection(population, scores)
            parent2 = tournament_selection(population, scores)
            child1 = crossover(parent1, parent2)
            child2 = crossover(parent1, parent2)
            child1 = mutate(child1, num_variables)
            child2 = mutate(child2, num_variables)
            if validate_tree(child1, num_variables):
                new_population.append(child1)
            if validate_tree(child2, num_variables):
                new_population.append(child2)

        population = new_population[:POPULATION_SIZE]

        # Population diversity check
        if len(set(str(ind) for ind in population)) < POPULATION_SIZE // 2:
            print("Low diversity in population, consider introducing new random individuals.")
            population.extend([generate_random_tree(3, num_variables) for _ in range(POPULATION_SIZE // 2)])
            population = population[:POPULATION_SIZE]  # Ensure population size remains constant

        # Print best fitness of the generation
        best_score = min(scores)
        print(f"Generation {generation}: Best fitness = {best_score}")

    scores = [fitness(ind, x, y) for ind in population]  # Recalculate scores for the final population
    best_index = scores.index(min(scores))
    return population[best_index]




if __name__ == "__main__":
    # Load the data
    x, y = load_data("problem_0.npz")

    # Run genetic programming
    best_tree = genetic_programming(x, y)

    # Output the best tree and its fitness
    print("Best expression:", best_tree)
    print("Best fitness:", fitness(best_tree, x, y))

Num variables: 2
Generation 0: Best fitness = 0.2567110770941986
Generation 1: Best fitness = 0.24725834335385383
Generation 2: Best fitness = 0.23794634599982795


C:\Users\silvi\AppData\Local\Temp\ipykernel_24952\2168546716.py:13: RuntimeWarning: divide by zero encountered in divide
  return a / b_safe


Generation 3: Best fitness = 0.23794634599982795
Generation 4: Best fitness = 0.011461833062336809
Generation 5: Best fitness = 0.05040207911046368
Generation 6: Best fitness = 0.23794634599982795
Generation 7: Best fitness = 0.23794634599982795


C:\Users\silvi\AppData\Local\Temp\ipykernel_24952\2168546716.py:62: RuntimeWarning: invalid value encountered in add
  return self.value(left_val, right_val)


Generation 8: Best fitness = 0.23794634599982795
Generation 9: Best fitness = 0.11931188383017033
Generation 10: Best fitness = 0.23794634599982795
Generation 11: Best fitness = 0.2985427259258043


C:\Users\silvi\AppData\Local\Temp\ipykernel_24952\2168546716.py:62: RuntimeWarning: invalid value encountered in multiply
  return self.value(left_val, right_val)


Generation 12: Best fitness = 0.23794634599982795
Generation 13: Best fitness = 0.18220992979179018
Generation 14: Best fitness = 0.009613944469277875
Generation 15: Best fitness = 0.09796874560808513
Generation 16: Best fitness = 0.09796874560808513
Generation 17: Best fitness = 0.23794634599982795
Generation 18: Best fitness = 0.23794634599982795
Generation 19: Best fitness = 0.23787201357589355
Generation 20: Best fitness = 0.1139944308288348
Generation 21: Best fitness = 0.22930993656789866
Generation 22: Best fitness = 0.35174990221643126
Generation 23: Best fitness = 0.6332661449780216
Generation 24: Best fitness = 0.6507392722616504
Generation 25: Best fitness = 0.23506074832798346
Generation 26: Best fitness = 0.21927510737125727
Generation 27: Best fitness = 0.11191133447787266
Generation 28: Best fitness = 0.23794634599982795
Generation 29: Best fitness = 0.23794634599982795
Generation 30: Best fitness = 0.08619261975442272
Generation 31: Best fitness = 0.23794634599982795
Ge